In [1]:
!pip install equiflow


In [2]:
import pandas as pd
from tableone import TableOne
import sklearn as sk
from xgboost import XGBClassifier, XGBRegressor
import numpy as np
from scipy.stats import ttest_ind, chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import chi2_contingency

from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.metrics import (
    roc_curve,
    roc_auc_score,
    average_precision_score,
    accuracy_score,
    recall_score,
    precision_score,
    f1_score,
    balanced_accuracy_score
)
from sklearn.utils import resample
from sklearn.metrics import r2_score, mean_absolute_error
from fairlearn.metrics import equalized_odds_ratio, demographic_parity_ratio
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
import warnings
import shap
from tableone import tableone
from matplotlib import pyplot as plt
import pydata_google_auth
import seaborn as sns
import pandas_gbq
from equiflow import EquiFlow

warnings.filterwarnings("ignore")


/Users/lnazer/anaconda3/lib/python3.11/site-packages/shap/plots/colors/_colorconv.py:819: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if np.issubdtype(dtype_in, np.dtype(dtype).type):
/Users/lnazer/anaconda3/lib/python3.11/site-packages/shap/plots/colors/_colorconv.py:819: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if np.issubdtype(dtype_in, np.dtype(dtype).type):
/Users/lnazer/anaconda3/lib/python3.11/site-packages/shap/plots/colors/_colorconv.py:819: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if np.issubdtype(dtype_in, np.dtype(dtype).type):
/Users/lnazer/anaconda3/lib/python3.11/site-packages/shap/plots/colors/_colorconv.py:819: DeprecationWarning: Converting `np

In [3]:
credentials = pydata_google_auth.get_user_credentials(['https://www.googleapis.com/auth/cloud-platform'],)
project = 'capacheiv'
pandas_gbq.context.credentials = credentials

In [4]:
query = ""
for fn in ['icustays', 'apache_vars', 'apache_pt_results', 'sepsis']:
    with open('eicu_1/'+fn+'.sql', 'r') as file:
        query += file.read()
query += "select * from sepsis"
project = 'capacheiv'

df_o = pd.read_gbq(query, dialect='standard', project_id=project)

In [5]:
query = ""
for fn in ['icustays', 'apache_vars', 'apache_pt_results', 'sepsis']:
    with open('eicu_2/'+fn+'.sql', 'r') as file:
        query += file.read()
query += "select * from sepsis"
project = 'capacheiv'

df_n = pd.read_gbq(query, dialect='standard', project_id=project)

In [65]:
df_n_no_duplicates = df_n[~df_n['uniquepid'].isin(df_o['uniquepid'])] 

In [66]:
df = pd.concat([df_o, df_n_no_duplicates], axis=0, ignore_index=True)

In [63]:
eicu1 = pd.read_csv("files/sepsis_adult_eicu_v1.csv")
eicu2 = pd.read_csv("files/sepsis_adult_eicu_v2.csv")

xai_df_sepsis = pd.concat([eicu1, eicu2], axis=0, ignore_index=True)

In [64]:
df = df.dropna(subset=['ethnicity'])
df = df[df['ethnicity'] != '']


In [67]:
df['ethnicity'] = df['ethnicity'].replace({'Black/African American': 'African American',
                                           '': 'Other/Unknown/Missing',
                                           'Native American':'Indigenous/Native',
                                           'Other/Unknown':'Other/Unknown/Missing'})
df['age_num'] = df['age_num'].astype(float)
df['ApacheScore'] = df['ApacheScore'].astype(float)
df['daysonvent'] = df['daysonvent'].astype(float)


df['gender'] = df['gender'].replace({ '': 'Unknown'})
df = df.fillna({'ethnicity':'Other/Unknown/Missing', 'gender':'Unknown'})
df = df.rename(columns = {'has_sepsis':'Sepsis', 'HOSP_NUM':'Number_of_Admissions', 'hospitaldischargeyear':'Discharge_Year',
                          'age_num':'Age', 'hosp_mortality':'hospital_mortality'})

df['set'] = df['Discharge_Year'].apply(
    lambda x: '2014-2015' if x in [2014, 2015] else(
              '2020-2021' if x in [2020, 2021] else np.nan
    )

)

In [69]:
columns = ['gender', 'Age', 'ethnicity', 'ApacheScore', 'Apache_Version',
           'predicted_hospital_mortality', 'mortality', 'Discharge_Year']

categorical = ['gender', 'ethnicity', 'Apache_Version', 'mortality', 'Discharge_Year']

table1 = TableOne(data=df,
                  columns=columns,
                  categorical=categorical,
                  groupby='hospital_mortality',  
                  isnull=True,
                  pval=True)  

table1

Grouped by hospital_mortality                                                      
                                                                                    Missing        Overall               0              1 P-Value
n                                                                                                   390509          221046          26595        
gender, n (%)                           Female                                            0  175664 (45.0)    99840 (45.2)   11950 (44.9)   0.039
                                        Male                                                 214561 (54.9)   121153 (54.8)   14631 (55.0)        
                                        Other                                                     10 (0.0)         7 (0.0)        1 (0.0)        
                                        Unknown                                                  274 (0.1)        46 (0.0)       13 (0.0)        
Age, mean (SD)                                                                          114    63.2 (16.9)     62.4 (17.0)    69.1 (14.6)  <0.001
ethnicity, n (%)                        African American                                  0   41688 (10.7)    25891 (11.7)    2820 (10.6)  <0.001
                                        Asian                                                   7206 (1.8)      3098 (1.4)      406 (1.5)        
                                        Caucasian                                            293481 (75.2)   166843 (75.5)   20037 (75.3)        
                                        Hispanic                                               14902 (3.8)      8628 (3.9)     1079 (4.1)        
                                        Indigenous/Native                                       3830 (1.0)      1390 (0.6)      169 (0.6)        
                                        Other/Unknown/Missing                                  29402 (7.5)     15196 (6.9)     2084 (7.8)        
ApacheScore, mean (SD)                                                               142868    55.2 (26.5)     51.6 (22.9)    85.2 (33.8)  <0.001
Apache_Version, n (%)                   IV                                           142868  100222 (40.5)    90938 (41.1)    9284 (34.9)  <0.001
                                        IVa                                                   97559 (39.4)    86588 (39.2)   10971 (41.3)        
                                        IVb                                                   49860 (20.1)    43520 (19.7)    6340 (23.8)        
predicted_hospital_mortality, mean (SD)                                              142868      0.0 (0.4)       0.0 (0.3)      0.2 (0.5)  <0.001
mortality, n (%)                        0                                                 0  363914 (93.2)  221046 (100.0)                 <0.001
                                        1                                                      26595 (6.8)                  26595 (100.0)        
Discharge_Year, n (%)                   2014                                              0   95513 (24.5)    64353 (29.1)    6457 (24.3)  <0.001
                                        2015                                                 105346 (27.0)    70442 (31.9)    7280 (27.4)        
                                        2020                                                  88297 (22.6)    44978 (20.3)    6172 (23.2)        
                                        2021                                                 101353 (26.0)    41273 (18.7)    6686 (25.1)        
[1] Chi-squared tests for the following variables may be invalid due to the low number of observations: gender.

In [70]:
df_sepsis = df[df.hospital_mortality.notna()]
df_sepsis = df_sepsis[df_sepsis.ApacheScore.notna()]
df_sepsis = df[(df['Sepsis'] == 1) | (df['patientunitstayid'].isin(xai_df_sepsis['patientunitstayid']))]
df_sepsis = df_sepsis[df_sepsis.Number_of_Admissions == 1]
df_sepsis = df_sepsis[df_sepsis.predicted_hospital_mortality >= 0]
df_sepsis = df_sepsis[df_sepsis.Apache_Version == 'IV']

In [71]:
EquiFlow._clean_missing = lambda self: None

eqfl = EquiFlow(
    data=df,
    initial_cohort_label='Admitted from 2014-2021',
    categorical=['gender', 'ethnicity', 'Sepsis', 'hospital_mortality']  
)


eqfl.add_exclusion(
    mask=(df.Number_of_Admissions == 1),
    exclusion_reason='Not First Admission',
    new_cohort_label=''
)

eqfl.add_exclusion(
    mask=(df['Sepsis'] == 1) | (df['patientunitstayid'].isin(xai_df_sepsis['patientunitstayid'])),
    exclusion_reason='Non-Septic Patients',
    new_cohort_label=''
)

eqfl.add_exclusion(
    mask=(df.Apache_Version == 'IV'),
    exclusion_reason='Has Apache IV score',
    new_cohort_label=''
)


eqfl.add_exclusion(
    mask=(df.ApacheScore.notna()),
    exclusion_reason='Missing APACHE Score',
    new_cohort_label=''
)


eqfl.add_exclusion(
    mask=(df.hospital_mortality.notna()),
    exclusion_reason='Missing Mortality Data',
    new_cohort_label=''
)

eqfl.add_exclusion(
    mask=(df.predicted_hospital_mortality >= 0),
    exclusion_reason='Predicted Hopsital\nMortality =-1',
    new_cohort_label=''
)





eqfl.plot_flows()

In [24]:
columns = ['gender', 'Age', 'ethnicity', 'ApacheScore', 'Apache_Version',
           'predicted_hospital_mortality', 'mortality', 'Discharge_Year']

categorical = ['gender', 'ethnicity', 'Apache_Version', 'mortality', 'Discharge_Year']

table1 = TableOne(data=df_sepsis,
                  columns=columns,
                  categorical=categorical,
                  groupby='hospital_mortality',  
                  isnull=True,
                  pval=True)  

table1

Grouped by hospital_mortality                                                    
                                                                                    Missing        Overall              0             1 P-Value
n                                                                                                    16165          13456          2709        
gender, n (%)                           Female                                            0    7780 (48.1)    6519 (48.4)   1261 (46.5)   0.143
                                        Male                                                   8382 (51.9)    6934 (51.5)   1448 (53.5)        
                                        Unknown                                                    3 (0.0)        3 (0.0)                      
Age, mean (SD)                                                                            0    64.6 (17.3)    63.5 (17.5)   70.1 (14.7)  <0.001
ethnicity, n (%)                        African American                                  0    1723 (10.7)    1463 (10.9)     260 (9.6)   0.343
                                        Asian                                                    233 (1.4)      191 (1.4)      42 (1.6)        
                                        Caucasian                                             12487 (77.2)   10377 (77.1)   2110 (77.9)        
                                        Hispanic                                                 726 (4.5)      610 (4.5)     116 (4.3)        
                                        Indigenous/Native                                         90 (0.6)       74 (0.5)      16 (0.6)        
                                        Other/Unknown/Missing                                    906 (5.6)      741 (5.5)     165 (6.1)        
ApacheScore, mean (SD)                                                                    0    64.2 (27.3)    59.7 (23.5)   87.1 (32.7)  <0.001
Apache_Version, n (%)                   IV                                                0  16165 (100.0)  13456 (100.0)  2709 (100.0)   1.000
predicted_hospital_mortality, mean (SD)                                                   0      0.2 (0.2)      0.2 (0.2)     0.4 (0.3)  <0.001
mortality, n (%)                        0                                                 0   13456 (83.2)  13456 (100.0)                <0.001
                                        1                                                      2709 (16.8)                 2709 (100.0)        
Discharge_Year, n (%)                   2014                                              0    6664 (41.2)    5580 (41.5)   1084 (40.0)  <0.001
                                        2015                                                   6333 (39.2)    5228 (38.9)   1105 (40.8)        
                                        2020                                                   2004 (12.4)    1725 (12.8)    279 (10.3)        
                                        2021                                                    1164 (7.2)      923 (6.9)     241 (8.9)        
[1] Chi-squared tests for the following variables may be invalid due to the low number of observations: gender.

In [25]:
def bootstrap_auroc_all(df, ethnicity, nboot=4000):
    df_eth = df[df['ethnicity'] == ethnicity]
    
    df_2014 = df_eth[df_eth['set'] == '2014-2015']
    df_2020 = df_eth[df_eth['set'] == '2020-2021']
    
    y_true_2014 = df_2014['mortality'].values
    y_pred_2014 = df_2014['predicted_hospital_mortality'].values
    
    y_true_2020 = df_2020['mortality'].values
    y_pred_2020 = df_2020['predicted_hospital_mortality'].values
    
    n_2014 = len(y_true_2014)
    n_2020 = len(y_true_2020)
    
    auroc_2014_list = []
    auroc_2020_list = []
    diffs = []
    
    for _ in range(nboot):
        idx_2014 = np.random.randint(0, n_2014, n_2014)
        idx_2020 = np.random.randint(0, n_2020, n_2020)
        
        y_true_2014_bs = y_true_2014[idx_2014]
        y_pred_2014_bs = y_pred_2014[idx_2014]
        
        y_true_2020_bs = y_true_2020[idx_2020]
        y_pred_2020_bs = y_pred_2020[idx_2020]
        
        if len(np.unique(y_true_2014_bs)) < 2 or len(np.unique(y_true_2020_bs)) < 2:
            continue 
        
        auroc_2014 = roc_auc_score(y_true_2014_bs, y_pred_2014_bs)
        auroc_2020 = roc_auc_score(y_true_2020_bs, y_pred_2020_bs)
        
        auroc_2014_list.append(auroc_2014)
        auroc_2020_list.append(auroc_2020)
        diffs.append(auroc_2020 - auroc_2014)
    
    return np.array(auroc_2014_list), np.array(auroc_2020_list), np.array(diffs)


In [26]:
results = []
for race in df_sepsis['ethnicity'].unique():
    auroc_2014_bs, auroc_2020_bs, diff_bootstrap = bootstrap_auroc_all(df_sepsis, race, nboot=4000)
    
    mean_diff = np.mean(diff_bootstrap)
    ci_lower = np.percentile(diff_bootstrap, 2.5)
    ci_upper = np.percentile(diff_bootstrap, 97.5)
    
    results.append({
        'ethnicity': race,
        'mean_diff_auroc': mean_diff,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'auroc_2014_bootstrap': auroc_2014_bs.tolist(),
        'auroc_2020_bootstrap': auroc_2020_bs.tolist(),
        'bootstrap_distribution': diff_bootstrap.tolist()
    })

auroc_diff_df = pd.DataFrame(results)

In [37]:
def bootstrap_smr_all(df, ethnicity, nboot=4000):
    df_eth = df[df['ethnicity'] == ethnicity]
    
    df_2014 = df_eth[df_eth['set'] == '2014-2015']
    df_2020 = df_eth[df_eth['set'] == '2020-2021']
    
    y_true_2014 = df_2014['mortality'].values
    y_pred_2014 = df_2014['predicted_hospital_mortality'].values
    
    y_true_2020 = df_2020['mortality'].values
    y_pred_2020 = df_2020['predicted_hospital_mortality'].values
    
    n_2014 = len(y_true_2014)
    n_2020 = len(y_true_2020)
    
    smr_2014_list = []
    smr_2020_list = []
    diffs = []
    
    for _ in range(nboot):
        idx_2014 = np.random.randint(0, n_2014, n_2014)
        idx_2020 = np.random.randint(0, n_2020, n_2020)
        
        y_true_2014_bs = y_true_2014[idx_2014]
        y_pred_2014_bs = y_pred_2014[idx_2014]
        
        y_true_2020_bs = y_true_2020[idx_2020]
        y_pred_2020_bs = y_pred_2020[idx_2020]
        
        obs_2014 = np.sum(y_true_2014_bs)
        exp_2014 = np.sum(y_pred_2014_bs)
        smr_2014 = obs_2014 / exp_2014 if exp_2014 > 0 else np.nan
        
        obs_2020 = np.sum(y_true_2020_bs)
        exp_2020 = np.sum(y_pred_2020_bs)
        smr_2020 = obs_2020 / exp_2020 if exp_2020 > 0 else np.nan
        
        if np.isnan(smr_2014) or np.isnan(smr_2020):
            continue
        
        smr_2014_list.append(smr_2014)
        smr_2020_list.append(smr_2020)
        diffs.append(smr_2020 - smr_2014)
    
    return np.array(smr_2014_list), np.array(smr_2020_list), np.array(diffs)


In [38]:
results = []
for race in df_sepsis['ethnicity'].unique():
    smr_2014_bs, smr_2020_bs, diff_bootstrap = bootstrap_smr_all(df_sepsis, race, nboot=4000)
    
    mean_diff = np.mean(diff_bootstrap)
    ci_lower = np.percentile(diff_bootstrap, 2.5)
    ci_upper = np.percentile(diff_bootstrap, 97.5)
    
    results.append({
        'ethnicity': race,
        'mean_diff_smr': mean_diff,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'smr_2014_bootstrap': smr_2014_bs.tolist(),
        'smr_2020_bootstrap': smr_2020_bs.tolist(),
        'bootstrap_distribution': diff_bootstrap.tolist()
    })

smr_diff_df = pd.DataFrame(results)


In [46]:
auroc_diff_df.to_pickle("auroc_diff_df.pkl")
smr_diff_df.to_pickle("smr_diff_df.pkl")

In [48]:
import pickle

# Replace 'your_file.pkl' with your file name
with open("auroc_diff_df.pkl", "rb") as f:
    data = pickle.load(f)

print(type(data))
# print(data)

<class 'pandas.core.frame.DataFrame'>


In [52]:
len(data['bootstrap_distribution'][0])

4000